In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
import numpy as np


In [ ]:
import read_iso

In [ ]:
cat = Table.read("../photometry/yasone2/allcolours.cat")
cat = cat[(cat["RFLAGS"] < 8 ) & (cat["RFLAGS_WEIGHT"] == 0) ]

In [ ]:
img = CCDData.read("../photometry/yasone2/stacked_r/coadd.fits", unit="adu")

In [ ]:
import arya

In [ ]:
# cat_gal = cat[~cat["LIKELY_STAR"]]
# cat = cat[cat["LIKELY_STAR"]]

In [ ]:
plt.scatter(cat["ALPHA_J2000"], cat["DELTA_J2000"], s=1)
plt.scatter(cat_gal["ALPHA_J2000"], cat_gal["DELTA_J2000"], s=0.3)

In [ ]:
isoname = "../MIST/MIST_v1.2_vvcrit0.4_SDSSugriz/MIST_v1.2_feh_m1.50_afe_p0.0_vvcrit0.4_SDSSugriz.iso.cmd"

In [ ]:
iso = read_iso.ISOCMD(isoname)

In [ ]:
iso = iso[np.log10(12e9)]

In [ ]:
ra0, dec0 = 262.34921, 6.42302

In [ ]:
coord0 = SkyCoord(ra0, dec0, unit=u.degree)
r_0 = 25 / 0.254 

In [ ]:
coord0.separation

In [ ]:
def to_coords(cat):
    return SkyCoord(cat["ALPHA_J2000"], cat["DELTA_J2000"], unit=u.degree)


In [ ]:
def select_centre(cat, coord0, radius):
    filt = coord0.separation(to_coords(cat)) < radius
    return cat[filt]

In [ ]:
def get_upper_radius(radius, radius_bkg):
    return np.sqrt(radius**2 + radius_bkg**2)


In [ ]:
def select_annuli(cat, coord0, radius, radius_bkg):
    radius_upper = get_upper_radius(radius, radius_bkg)
    # print(radius**2, radius_upper**2 - radius_bkg**2)
    seps = coord0.separation(to_coords(cat))

    filt = seps >= radius_bkg
    filt &= seps < radius_upper
    return cat[filt]

In [ ]:
coord0.ra.to("rad").value

In [ ]:
def plot_annuli(radius, radius_bkg):
    circ = plt.Circle((0, 0), radius, color="black", lw=1, fill=False)
    circ2 = plt.Circle((0, 0), radius_bkg, color="black", lw=1, fill=False)
    circ3 = plt.Circle((0, 0), get_upper_radius(radius, radius_bkg), color="black", lw=1, fill=False)
    ax = plt.gca()
        
    ax.add_patch(circ)
    ax.add_patch(circ2)
    ax.add_patch(circ3)

In [ ]:
def to_tangent(coord, coord0):
    alpha = coord.ra.to("rad").value
    delta = coord.dec.to("rad").value
    alpha0 = coord0.ra.to("rad").value
    delta0 = coord0.dec.to("rad").value


    denom = np.sin(delta) * np.sin(delta0) + np.cos(delta) * np.cos(delta0) * np.cos(alpha - alpha0)

    xi_num = np.cos(delta) * np.sin(alpha - alpha0)
    eta_num = np.sin(delta) * np.cos(delta0) - np.cos(delta) * np.sin(delta0) * np.cos(alpha-alpha0) 

    xi = np.rad2deg(xi_num/denom)
    eta = np.rad2deg(eta_num / denom)
    return xi, eta

In [ ]:
xi, eta = to_tangent(to_coords(cat), coord0)

In [ ]:
cat["xi"] = xi * 60 * u.arcmin
cat["eta"] = eta * 60 * u.arcmin

In [ ]:
r_h = 25 / 60

In [ ]:
r_inner = 40  / 60
r_ring = 2*r_inner

In [ ]:
sample_bkg = select_annuli(cat, coord0, r_inner * u.arcmin, r_ring*u.arcmin)

In [ ]:
sample_cen = select_centre(cat, coord0, r_inner * u.arcmin )

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect(1)
cat_star = cat[cat["LIKELY_STAR"]]
plt.scatter(cat["xi"], cat["eta"], s=0.3, color="k")

plt.scatter(cat_star["xi"], cat_star["eta"], s=0.5, color=arya.COLORS[1])
plt.xlim(-4, 4)
plt.ylim(-4, 4)

In [ ]:
def plot_iso(dm=10, A=0.1, phase_max=3):
    filt = iso["phase"] < phase_max
    plt.plot(iso["SDSS_g"][filt] - iso["SDSS_r"][filt] + A, iso["SDSS_g"][filt] + dm)

In [ ]:
def plot_iso_ri(dm=10, A=0.1, phase_max=3):
    filt = iso["phase"] < phase_max
    plt.plot(iso["SDSS_r"][filt] - iso["SDSS_i"][filt] + A, iso["SDSS_r"][filt] + dm)

In [ ]:
coord0

In [ ]:
import seaborn as sns

In [ ]:
def to_pandas(tbl):
    names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
    return tbl[names].to_pandas()

In [ ]:
cmd_selection = cat["RMAG"] - cat["IMAG"] > -0.2
cmd_selection &= cat["RMAG"] - cat["IMAG"] <0.0

In [ ]:
cat_cmd = cat[cmd_selection]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(4, 4))

plt.sca(ax[0][0])
ax[0][0].set_aspect(1)
plt.scatter(cat["xi"], cat["eta"], s=1)
plot_annuli(r_inner, r_ring)
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=1)
plt.scatter(sample_bkg["xi"], sample_bkg["eta"], s=1)
plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.sca(ax[0][1])
sns.kdeplot(to_pandas(cat[cmd_selection]), x="xi", y="eta",)
plt.xlim(-3, 3)
plt.ylim(-3, 3)

plt.sca(ax[1][0])
plt.scatter(cat["GMAG"] - cat["RMAG"], cat["GMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_cen["GMAG"] - sample_cen["RMAG"], sample_cen["GMAG"], fmt="o", label="centre")

plot_iso(dm=16.5, A=0.45)

plt.xlim(-0.5, 2.5)
plt.ylim(27, 17)


plt.sca(ax[1][1])
plt.scatter(cat["GMAG"] - cat["RMAG"], cat["GMAG"], s=0.3, lw=0, color="grey")

plt.errorbar(sample_bkg["GMAG"] - sample_bkg["RMAG"], sample_bkg["GMAG"], fmt="o", label="background annulus")

plot_iso(dm=16.5, A=0.45)
plt.xlim(-0.5, 2.5)
plt.ylim(27, 17)



In [ ]:
plt.scatter(cat_cmd["RMAG"] - cat_cmd["IMAG"], cat_cmd["RMAG"], )


In [ ]:
fig = plt.figure(figsize=(6, 5))
Nr, Nc = 3, 3

fig.add_subplot(Nr, Nc, 1)
plt.gca().set_aspect(1)
plt.scatter(cat["xi"], cat["eta"], s=1, color="grey")
plot_annuli(r_inner, r_ring)
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=1, color=arya.COLORS[2])
plt.scatter(sample_bkg["xi"], sample_bkg["eta"], s=1, color=arya.COLORS[1])
plt.xlim(3, -3)
plt.ylim(-3, 3)

fig.add_subplot(Nr, Nc, 2)
sns.kdeplot(to_pandas(cat[cmd_selection]), x="xi", y="eta", bw_adjust=0.5)
plt.scatter(cat_cmd["xi"], cat_cmd["eta"], s=1, color=arya.COLORS[2])
plt.gca().set_aspect(1)
plt.xlim(3, -3)
plt.ylim(-3, 3)

fig.add_subplot(Nr, Nc, 3, projection=img.wcs)
plt.imshow(img.data, origin="lower", norm="log")
plt.scatter(cat["X_IMAGE"], cat["Y_IMAGE"], s=0.3, lw=0, color=arya.COLORS[2])

fig.add_subplot(Nr, Nc, 4)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")
plt.errorbar(sample_cen["RMAG"] - sample_cen["IMAG"], sample_cen["RMAG"], fmt="o", label="centre", color=arya.COLORS[2])

plot_iso_ri(dm=16.5, A=-0.3)

plt.xlim(-1, 1)
plt.ylim(27, 17)


fig.add_subplot(Nr, Nc, 5)
plt.scatter(cat["RMAG"] - cat["IMAG"], cat["RMAG"], s=0.3, lw=0, color="grey")

plt.errorbar(sample_bkg["RMAG"] - sample_bkg["IMAG"], sample_bkg["RMAG"], fmt="o", label="background annulus", color=arya.COLORS[1])

plot_iso_ri(dm=16.5, A=-0.3)
plt.xlim(-1, 1)
plt.ylim(27, 17)



### 

In [ ]:
def gmag_err(gmag):
    return 0.2 + np.minimum(0.0025 + np.exp(gmag - 27.75), 1)

In [ ]:
import shapely

In [ ]:
def make_polygon(iso, mag_err, dm, A=0, yfilt="SDSS_r", xfilt="SDSS_i"):
    filt = iso["phase"] < 3

    x = iso[yfilt][filt].data - iso[xfilt][filt].data + A
    y = iso[yfilt][filt].data + dm

    x_min = x - gmag_err(y)
    x_max = x + gmag_err(y)

    return np.hstack([x_min, x_max[::-1], x_min[0]]), np.hstack([y, y[::-1], y[0]])


In [ ]:
def is_in_poly(x, y, x_poly, y_poly):
    N = len(x)
    poly = shapely.Polygon(np.vstack([x_poly, y_poly]).T)
    
    in_poly = np.full(N, False)
    for i in range(N):
        if np.isfinite(x[i]) and np.isfinite(y[i]):
            point = shapely.Point(x[i], y[i])
            in_poly[i] = point.within(poly)

    return in_poly

In [ ]:
x_poly, y_poly = make_polygon(iso, gmag_err, dm=16.6, A=0.0)

In [ ]:
cmd_filt = is_in_poly(cat["RMAG"].data - cat["IMAG"].data, cat["RMAG"].data, x_poly, y_poly)

In [ ]:
np.sum(cmd_filt)

In [ ]:

plt.scatter(cat["RMAG"][cmd_filt] - cat["IMAG"][cmd_filt], cat["RMAG"][cmd_filt])
plt.scatter(cat["RMAG"][~cmd_filt] - cat["IMAG"][~cmd_filt], cat["RMAG"][~cmd_filt], s=1)
plot_iso_ri(dm=16.5, A=0)
plt.plot(x_poly, y_poly, color="k")
plt.ylim(27, 16)
plt.xlim(-1, 2)

In [ ]:
plt.scatter(cat["xi"][cmd_filt], cat["eta"][cmd_filt])

In [ ]:
sns.kdeplot(to_pandas(cat[cmd_filt]), x="xi", y="eta", bw_adjust=0.5)
plt.xlim(-3, 3)
plt.ylim(-3, 3)

In [ ]:
plt.plot([0,3], [0,3])

plt.scatter(np.exp(cat["GMAG"] - 27.9), cat["GMAG_ERR"], s=0.1, lw=0, alpha=0.2, color="k")
plt.xlim(0, 1)
plt.ylim(0, 1)

In [ ]:
plt.scatter(cat["GMAG"], cat["GMAG_ERR"] / ( 0.0025 + np.exp(cat["GMAG"] - 27.75)), s=0.3, lw=0, alpha=0.1, color="k")
# plt.xlim(0, 1e3)
plt.ylim(0, 2)